In [64]:
import numpy as np
import glob
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.signal import find_peaks, savgol_filter
from scipy.integrate import trapezoid
from scipy.stats import linregress

import pandas as pd
import scipy
import datetime
import time
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import re
import os

from hplc.io import load_chromatogram
from hplc.quant import Chromatogram

import plotly.graph_objs as go
# from plotly.events import plotly_click

from ipywidgets import VBox, Button, Output


In [65]:
import dash
from dash import dcc, html, Input, Output, State
import dash_bootstrap_components as dbc
import plotly.express as px

import pandas as pd
import math


## functions

In [66]:
def readfile(path):
    df=pd.read_csv(path, delimiter='\t',  skiprows=[1,2,3,4],  index_col=False)
    df.rename(columns=lambda x: re.sub(' ','',x), inplace=True)  #sometimes the software puts extra spaces to the columns names
    df.rename(columns={'TraceforMass:':'time', 'Li7(MR)':'Li7', 'S32(MR)':'intensity', 'Ni64++(MR)':'BL32',
                      'Cl35(MR)':'Cl35', 'Ar40(MR)':'Ar40',
                       'P31(MR)':'P31', 'Ni62++(MR)':'BL31',
                       'Mn55(MR)':'Mn55', 'Ni58(MR)':'Ni58',
                      }, inplace=True)
    df.time /= 60
    return df

### last version 

## 251007

In [71]:
# ---------- SETTINGS ----------
SPECTRUM_FOLDER = "..\\..\\..\\..\\..\\data Element XR\\251002*\\" 

# Get list of available files
file_list = glob.glob(os.path.join(".\\..\\..\\..\\..\\data Element XR\\251002*\\*.txt"))

# Prepare dropdown options
dropdown_options = [
    {"label": os.path.basename(f), "value": f} for f in file_list
]
# dropdown_options

In [73]:
# ----------- APP SETUP -------------
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.FLATLY])

# Styles for active/inactive buttons
active_style = {
    "backgroundColor": "#2b8a3e",
    "color": "white",
    "fontWeight": "bold",
    "border": "2px solid #1c6b2a",
}
inactive_style = {
    "backgroundColor": "#f0f0f0",
    "color": "black",
    "fontWeight": "normal",
    "border": "1px solid #ccc",
}

app.layout = dbc.Container(
    [
        html.H3("Interactive Spectrum Editor", className="mt-3"),

        # ---- File selection ----
        dbc.Row([
            dbc.Col(html.Label("Select Spectrum File:"), width="auto"),
            dbc.Col(
                dcc.Dropdown(
                    id="spectrum-selector",
                    options=dropdown_options,
                    placeholder="Select a file...",
                    clearable=True,
                ),
                width=6
            ),
        ], className="mb-4"),
        
        dbc.Row([
            dbc.Col(html.Button("Add Boundaries", id="add-points-btn", n_clicks=0, style=inactive_style)),
            dbc.Col(html.Button("Add Peaks", id="add-peaks-btn", n_clicks=0, style=inactive_style)),
            dbc.Col(html.Button("Modify", id="modify-btn", n_clicks=0, style=inactive_style)),
            dbc.Col(html.Button("Add Peaks by Dragging", id="drag-add-btn", style=inactive_style)),
            dbc.Col(html.Button("Clear all", id="clear-btn", n_clicks=0, style={"marginLeft": "10px", "backgroundColor": "#d9534f", "color": "white"})),           
            dbc.Col(html.Button("Integrate", id="integrate-btn", n_clicks=0, style=inactive_style)),
        ], className="mb-3", justify="start"),

        dcc.Graph(id="spectrum-plot", clear_on_unhover=True),

        # html.Div(id="integration-results", className="mt-3 text-primary fw-bold"),

        # Store mode states and data
        dcc.Store(id="add-mode", data=False),
        dcc.Store(id="modify-mode", data=False),
        dcc.Store(id="add-peaks-mode", data=False),
        dcc.Store(id="added-points", data=[]),
        dcc.Store(id="added-peaks", data=[]),
        dcc.Store(id="spectrum-data", data=None),
        dcc.Store(id="drag-add-mode", data=False),
        dcc.Store(id="integration-results", data=None),
    ],
    fluid=True
)

# ----------- CALLBACKS -------------

# Load the selected spectrum file
@app.callback(
    Output(component_id="spectrum-data", component_property="data"),
    Input("spectrum-selector", "value"),
)
def load_spectrum(file_path):
    if not file_path:
        raise dash.exceptions.PreventUpdate
    try:
        df = readfile(file_path)     
        # df.rename(columns={'S32':'intensity'})
        
        if not {"time", "intensity"}.issubset(df.columns):
            print("no data")
            return dash.no_update
        return df.to_dict("records")
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return dash.no_update
        
# Toggle mode logic — only one active at a time
@app.callback(
    Output("add-mode", "data"),
    Output("modify-mode", "data"),
    Output("add-peaks-mode", "data"),
    Output("drag-add-mode", "data"),
    Input("add-points-btn", "n_clicks"),
    Input("modify-btn", "n_clicks"),
    Input("add-peaks-btn", "n_clicks"),
    Input("drag-add-btn", "n_clicks"),
    prevent_initial_call=True,
)
def toggle_modes(add_click, modify_click, peaks_click, drag_click):
    ctx = dash.callback_context
    if not ctx.triggered:
        raise dash.exceptions.PreventUpdate

    clicked = ctx.triggered[0]["prop_id"].split(".")[0]

    return (
        clicked == "add-points-btn",
        clicked == "modify-btn",
        clicked == "add-peaks-btn",
        clicked == "drag-add-btn",
    )

# Button visual highlighting
@app.callback(
    Output("add-points-btn", "style"),
    Output("modify-btn", "style"),
    Output("add-peaks-btn", "style"),
    Output("drag-add-btn", "style"),
    Input("add-mode", "data"),
    Input("modify-mode", "data"),
    Input("add-peaks-mode", "data"),
    Input("drag-add-mode", "data"),
)
def update_button_styles(add_mode, modify_mode, peaks_mode, drag_mode):
    if add_mode:
        return active_style, inactive_style, inactive_style, inactive_style
    elif modify_mode:
        return inactive_style, active_style, inactive_style, inactive_style
    elif peaks_mode:
        return inactive_style, inactive_style, active_style, inactive_style
    elif drag_mode:
        return inactive_style, inactive_style, inactive_style, active_style
    else:
        return inactive_style, inactive_style, inactive_style, inactive_style


# Handle clicks for adding/removing points and peaks
@app.callback(
    Output("added-points", "data"),
    Output("added-peaks", "data"),
    Input("spectrum-plot", "clickData"),
    State("add-mode", "data"),
    State("modify-mode", "data"),
    State("add-peaks-mode", "data"),
    State("added-points", "data"),
    State("added-peaks", "data"),
    prevent_initial_call=True,
)
def handle_click(clickData, add_mode, modify_mode, peaks_mode, added_points, added_peaks):
    if clickData is None:
        raise dash.exceptions.PreventUpdate

    point = clickData["points"][0]
    x, y = point["x"], point["y"]
    curve = point["curveNumber"]

    if add_mode and not modify_mode and not peaks_mode:
        added_points.append((x, y))

    elif modify_mode and not add_mode and not peaks_mode:
        new_points = [
            (px, py) for (px, py) in added_points
            if not (math.isclose(px, x, rel_tol=1e-6, abs_tol=1e-6) and
                    math.isclose(py, y, rel_tol=1e-6, abs_tol=1e-6))
        ]
        new_peaks = [
            (px, py) for (px, py) in added_peaks
            if not (math.isclose(px, x, rel_tol=1e-6, abs_tol=1e-6)
                    and math.isclose(py, y, rel_tol=1e-6, abs_tol=1e-6))
        ]
        added_points, added_peaks = new_points, new_peaks

    elif peaks_mode and not add_mode and not modify_mode and curve == 0:
        added_peaks.append((x, y))

    return added_points, added_peaks


@app.callback(
    Output("added-points", "data", allow_duplicate=True),
    Output("added-peaks", "data", allow_duplicate=True),
    Input("spectrum-plot", "selectedData"),
    State("drag-add-mode", "data"),
    State("added-points", "data"),
    State("added-peaks", "data"),
    State("spectrum-data", "data"),
    prevent_initial_call=True,
)
def handle_drag_select(selectedData, drag_mode, added_points, added_peaks, spectrum_data):
    if not drag_mode or selectedData is None:
        raise dash.exceptions.PreventUpdate

    df = pd.DataFrame(spectrum_data)
    
    if not {"time", "intensity"}.issubset(df.columns):
        raise dash.exceptions.PreventUpdate
    # Get x boundaries of selection
    xs = [p for p in selectedData[list(selectedData.keys())[1]]["x"]]

    x_min, x_max = min(xs), max(xs)

    # Get the spectrum points near the boundaries
    left_idx = (df["time"] - x_min).abs().idxmin()
    right_idx = (df["time"] - x_max).abs().idxmin()

    left_pt = (float(df.loc[left_idx, "time"]), float(df.loc[left_idx, "intensity"]))
    right_pt = (float(df.loc[right_idx, "time"]), float(df.loc[right_idx, "intensity"]))
    added_points.extend([left_pt, right_pt])

    # Find the max intensity between boundaries
    mask = (df["time"] >= left_pt[0]) & (df["time"] <= right_pt[0])
    sub_df = df[mask]
    if not sub_df.empty:
        peak_idx = sub_df["intensity"].idxmax()
        peak_pt = (float(df.loc[peak_idx, "time"]), float(df.loc[peak_idx, "intensity"]))
        added_peaks.append(peak_pt)

    return added_points, added_peaks

# === Clear all added points and peaks ===
@app.callback(
    Output("added-points", "data", allow_duplicate=True),
    Output("added-peaks", "data", allow_duplicate=True),
    Input("clear-btn", "n_clicks"),
    prevent_initial_call=True,
)
def clear_all(n):
    return [], []


# === INTEGRATION CALLBACK ===
@app.callback(
    Output("integration-results", "data"),
    Input("integrate-btn", "n_clicks"),
    State("spectrum-data", "data"),
    State("added-peaks", "data"),
    State("added-points", "data"),
    prevent_initial_call=True,
)
def integrate_peaks(n_clicks, spectrum_data, added_peaks, added_points):
    if spectrum_data is None and not added_peaks or not added_points:
        return "⚠️ Add at least one peak and two boundary points first."

    results = []
    df_results = pd.DataFrame(columns=["rt","area"])
    df = pd.DataFrame(spectrum_data)
    added_points_sorted = sorted(added_points, key=lambda p: p[0])

    for (x_peak, _) in added_peaks:
        # find nearest left/right boundary points
        left_points = [p for p in added_points_sorted if p[0] < x_peak]
        right_points = [p for p in added_points_sorted if p[0] > x_peak]
        if not left_points or not right_points:
            continue
        left = left_points[-1][0]
        right = right_points[0][0]

        # restrict data to between left and right
        mask = (df["time"] >= left) & (df["time"] <= right)
        x_seg = df.loc[mask, "time"]
        y_seg = df.loc[mask, "intensity"]
        if len(x_seg) > 1:
            area = trapezoid(y_seg, x_seg) - (right - left) * (left_points[-1][1] + right_points[0][1]) / 2 #substracting trpz of the peak base
            df_results.loc[len(df_results)]= [x_peak, area]
        df_results.drop_duplicates(subset=['rt'], inplace=True)    #remove lines with the same rt
        df_results.sort_values(by=["rt"], inplace=True, ignore_index=True)
    if area:
        return display(df_results)
    else:
        return "⚠️ Could not find valid boundaries for integration."

# === PLOT CALLBACK ===
@app.callback(
    Output("spectrum-plot", "figure"),
    Input("spectrum-data", "data"),
    Input("added-points", "data"),
    Input("added-peaks", "data"),
)
def update_plot(spectrum_data, added_points, added_peaks):
    fig = go.Figure()
    
    if spectrum_data is None:
        fig.update_layout(
            title="Select a spectrum file to display",
            template="simple_white",
            height=500,
        )
        return fig

    df = pd.DataFrame(spectrum_data)
    
    # Spectrum line
    fig.add_scatter(
        x=df["time"], y=df["intensity"],
        mode="lines", name="Spectrum",
        line=dict(color="black")
    )
    # Added points
    if added_points:
        added_points = sorted(added_points, key=lambda p: p[0])
        x_p, y_p = zip(*added_points)
        
        fig.add_trace(go.Scatter(
            x=x_p, y=y_p,
            mode="lines+markers", name="Added Points",
            marker=dict(color="red", size=10, symbol="circle"),
        ))

    # Added peaks
    if added_peaks:
        x_peak, y_peak = zip(*added_peaks)
        fig.add_trace(go.Scatter(
            x=x_peak, y=y_peak,
            mode="markers", name="Added Peaks",
            marker=dict(color="blue", size=12, symbol="star"),
        ))

    fig.update_layout(
        template="simple_white",
        dragmode="select",
        clickmode="event+select",
        hovermode="closest",
        height=400,
        margin=dict(l=40, r=40, t=40, b=40),
    )
    return fig


if __name__ == "__main__":
    app.run(debug=True)

,rt,area
0,1.061567,1.389321e+06
1,2.053817,2.101577e+07
2,3.965717,2.450623e+07
3,8.370333,5.984456e+06
4,15.969517,2.733076e+03
5,17.155367,5.821477e+05
6,35.620898,1.306063e+04
7,40.025517,2.705646e+05


,rt,area
0,0.38395,7.791993e+07


,rt,area
0,0.38395,7.791993e+07


,rt,area
0,6.247617,2.529302e+06


## 2

In [17]:
import dash
from dash import dcc, html, Input, Output, State
import plotly.graph_objs as go
import pandas as pd

# Example spectrum data
time = np.linspace(0, 10, 50)
intensity = np.exp(-(time - 3) ** 2) + 0.5 * np.exp(-(time - 7) ** 2 / 0.5)
df = pd.DataFrame({"time": time, "intensity": intensity})

app = dash.Dash(__name__)

app.layout = html.Div([
    html.Div([
        html.Button("Add Points: OFF", id="add-btn", n_clicks=0),
        html.Button("Modify: OFF", id="modify-btn", n_clicks=0),
    ]),
    dcc.Graph(id="spectrum-plot", config={"displayModeBar": True}),
    dcc.Store(id="added-points", data=[]),  # store added points
    dcc.Store(id="add-mode", data=False),
    dcc.Store(id="modify-mode", data=False),
])

@app.callback(
    Output("add-mode", "data"),
    Output("add-btn", "children"),
    Input("add-btn", "n_clicks"),
    State("add-mode", "data"),
)
def toggle_add(n_clicks, mode):
    if n_clicks == 0:
        raise dash.exceptions.PreventUpdate
    new_mode = not mode
    return new_mode, f"Add Points: {'ON' if new_mode else 'OFF'}"

@app.callback(
    Output("modify-mode", "data"),
    Output("modify-btn", "children"),
    Input("modify-btn", "n_clicks"),
    State("modify-mode", "data"),
)
def toggle_modify(n_clicks, mode):
    if n_clicks == 0:
        raise dash.exceptions.PreventUpdate
    new_mode = not mode
    return new_mode, f"Modify: {'ON' if new_mode else 'OFF'}"

@app.callback(
    Output("added-points", "data"),
    Input("spectrum-plot", "clickData"),
    State("add-mode", "data"),
    State("modify-mode", "data"),
    State("added-points", "data"),
    prevent_initial_call=True,
)
def handle_click(clickData, add_mode, modify_mode, added_points):
    if clickData is None:
        raise dash.exceptions.PreventUpdate

    point = clickData["points"][0]
    x, y = point["x"], point["y"]
    curve = point["curveNumber"]  # tells which trace was clicked

    if add_mode and not modify_mode:
        # Add point only when clicking on the base spectrum
        if (x, y) not in added_points:
            added_points.append((x, y))
            print(x,y)

    elif modify_mode and not add_mode and curve == 1:
        # remove clicked point from added_points (with tolerance)
        new_points = []
        for (px, py) in added_points:
            if not (math.isclose(px, x, rel_tol=1e-6, abs_tol=1e-6) and
                    math.isclose(py, y, rel_tol=1e-6, abs_tol=1e-6)):
                new_points.append((px, py))
        added_points = new_points

    return added_points

@app.callback(
    Output("spectrum-plot", "figure"),
    Input("added-points", "data"),
)
def update_graph(added_points):
    fig = go.Figure()

    # Original spectrum
    fig.add_trace(go.Scatter(
        x=df["time"],
        y=df["intensity"],
        mode="lines",
        name="Spectrum"
    ))

    # Added points
    if added_points:
        xs, ys = zip(*added_points)
        fig.add_trace(go.Scatter(
            x=xs,
            y=ys,
            mode="markers",
            name="Added Points",
            marker=dict(size=12, color="red", symbol="circle-open")
        ))

    fig.update_layout(clickmode="event+select")
    return fig

if __name__ == "__main__":
    app.run(debug=True)

2.4489795918367347 0.7381387305946908
3.4693877551020407 0.8022581303328848


In [5]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output, State, callback_context

# ----------------------------
# Example spectrum dataframe
# ----------------------------
time = np.linspace(0, 10, 500)
intensity = np.exp(-(time - 3) ** 2) + 0.5 * np.exp(-(time - 7) ** 2 / 0.5)
spectrum = pd.DataFrame({"time": time, "intensity": intensity})

# Array to store clicked points
clicked_points = []

# ----------------------------
# Helper: build spectrum figure
# ----------------------------
def make_figure():
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=spectrum["time"], y=spectrum["intensity"],
        mode="lines", name="spectrum"
    ))

    if clicked_points:
        xs, ys = zip(*clicked_points)
        fig.add_trace(go.Scatter(
            x=xs, y=ys,
            mode="markers+lines",
            name="added points",
            marker=dict(color="red", size=10, symbol="x"),
            line=dict(color="red", width=2)
        ))

    fig.update_layout(
        title="Spectrum (Add/Modify Points)",
        xaxis_title="Time",
        yaxis_title="Intensity",
        clickmode="event+select"
    )
    return fig

# ----------------------------
# Dash app
# ----------------------------
app = Dash(__name__)

app.layout = html.Div([
    html.H3("Interactive Spectrum"),
    dcc.Graph(id="spectrum-graph", figure=make_figure()),
    html.Button("Add Points (OFF)", id="add-btn", n_clicks=0, style={"margin": "10px"}),
    html.Button("Modify Points (OFF)", id="modify-btn", n_clicks=0, style={"margin": "10px"}),
    html.Button("Clear Points", id="clear-btn", n_clicks=0, style={"margin": "10px"}),
    html.Div(id="points-list"),
    dcc.Store(id="add-mode", data=False),
    dcc.Store(id="modify-mode", data=False)
])

# ----------------------------
# Toggle Add Mode
# ----------------------------
@app.callback(
    Output("add-mode", "data"),
    Output("add-btn", "children"),
    Input("add-btn", "n_clicks"),
    State("add-mode", "data"),
    prevent_initial_call=True
)
def toggle_add_mode(n_clicks, add_mode):
    new_mode = not add_mode
    button_text = "Add Points (ON)" if new_mode else "Add Points (OFF)"
    return new_mode, button_text

# ----------------------------
# Toggle Modify Mode
# ----------------------------
@app.callback(
    Output("modify-mode", "data"),
    Output("modify-btn", "children"),
    Input("modify-btn", "n_clicks"),
    State("modify-mode", "data"),
    prevent_initial_call=True
)
def toggle_modify_mode(n_clicks, modify_mode):
    new_mode = not modify_mode
    button_text = "Modify Points (ON)" if new_mode else "Modify Points (OFF)"
    return new_mode, button_text

# ----------------------------
# Update graph
# ----------------------------
@app.callback(
    Output("spectrum-graph", "figure"),
    Output("points-list", "children"),
    Input("spectrum-graph", "clickData"),
    Input("clear-btn", "n_clicks"),
    State("add-mode", "data"),
    State("modify-mode", "data"),
    prevent_initial_call=True
)
def update_graph(clickData, clear_clicks, add_mode, modify_mode):
    global clicked_points

    ctx = callback_context
    trigger_id = ctx.triggered[0]["prop_id"].split(".")[0] if ctx.triggered else None

    # Clear points
    if trigger_id == "clear-btn":
        clicked_points = []

    # Add point if Add Mode is ON
    elif trigger_id == "spectrum-graph" and clickData is not None and add_mode:
        x_clicked = clickData["points"][0]["x"]
        y_clicked = clickData["points"][0]["y"]
        clicked_points.append((x_clicked, y_clicked))

    # elif trigger_id == "modify-mode":
    #     # Remove closest point if exists
    #     if x_clicked:
    #         distances = [(x_clicked - x)**2 + (y_clicked - y)**2 for x, y in zip(clicked_points['x'], clicked_points['y'])]
    #         min_idx = distances.index(min(distances))
    #         if min(distances) < 0.1:  # threshold for deletion
    #             added_points['x'].pop(min_idx)
    #             added_points['y'].pop(min_idx)
                
    fig = make_figure()
    points_text = [f"({round(x,3)}, {round(y,3)})" for x, y in clicked_points]
    return fig, html.Ul([html.Li(pt) for pt in points_text])

# ----------------------------
# Run server
# ----------------------------
if __name__ == "__main__":
    app.run(debug=True)
